In [2]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

In [3]:
df = pd.read_hdf('data4.hdf', 'table')
df.head(5)

,time,voltage,amplitude
0,-0.015000,0.644351,0.3
1,-0.014999,0.577405,0.3
2,-0.014998,0.711297,0.3
3,-0.014998,0.644351,0.3
4,-0.014997,0.711297,0.3


In [5]:
N = 1024

a, b = (23287, 54485)

t0 = df[df['amplitude'] == .3]['time'][a]
t1 = df[df['amplitude'] == .3]['time'][b]
df = df[(df['time'] > t0) & ((df['time'] < t1))]

In [9]:
f0, f1 = (90e6, 110e6)

def time_to_freq(t):
    return f0 + (f1 - f0) * (t - t0) / (t1 - t0)

def downscale(x, N):
    i = np.linspace(0, 1, len(x))
    j = np.linspace(0, 1, N)

    return np.interp(j, i, x)

In [142]:
import torch

from torch import optim
from torch import nn
from torch.nn import functional as F

class Net(nn.Module):
    
    def __init__(self, num_features, num_hidden, num_outputs):
        super().__init__()

        self.lin1 = nn.Linear(num_features, num_hidden)
        self.lin2 = nn.Linear(num_hidden, num_outputs)
        
        torch.nn.init.xavier_uniform_(self.lin1.weight)
        torch.nn.init.xavier_uniform_(self.lin2.weight)
        
    def forward(self, x):
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        
        return x
    
net = Net(2, 64, 1).double()

optimizer = optim.Adam(net.parameters(), lr=.2)

In [14]:
U = df[df['amplitude'] == df['amplitude'].max()]['voltage'].min()
U

2.7866105856

In [134]:
frequencies = []
amplitudes = []
voltages = []

for A in df['amplitude'].unique():
    s = df['amplitude'] == A
    
    f = downscale(time_to_freq(df[s]['time']), N)
    u = downscale(df[s]['voltage'], N)
    
    frequencies.append(f)
    voltages.append(u)
    amplitudes.append(A*np.ones(N))
    
inputs = np.stack([
    np.array(frequencies).flatten(),
    np.array(voltages).flatten()
])

targets = np.array(amplitudes).flatten()

inputs.shape, targets.shape

((2, 716800), (716800,))

In [150]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

dataset = TensorDataset(torch.from_numpy(inputs).t(), torch.from_numpy(targets).unsqueeze(1))

In [151]:
for epoch in range(10000):
    loader = DataLoader(dataset, batch_size=10, shuffle=True)
    
    for i, (x, y) in enumerate(loader):
        optimizer.zero_grad()
    
        loss = F.mse_loss(net(x), y)
        loss.backward()
    
        optimizer.step()
    
        if i % 10000 == 0:
            print(f'epoch: {epoch}, iteration: {i}, loss: {loss.item()}')

epoch: 0, iteration: 0, loss: 0.04211641855342204
epoch: 0, iteration: 10000, loss: 0.036178568323850545
epoch: 0, iteration: 20000, loss: 0.04209824055164772
epoch: 0, iteration: 30000, loss: 0.044495493399918644
epoch: 0, iteration: 40000, loss: 0.03491926767755383
epoch: 0, iteration: 50000, loss: 0.0686844024798204
epoch: 0, iteration: 60000, loss: 0.03946907564699595
epoch: 0, iteration: 70000, loss: 0.05996274145865982
epoch: 1, iteration: 0, loss: 0.039151614669733145
epoch: 1, iteration: 10000, loss: 0.05900398183138651
epoch: 1, iteration: 20000, loss: 0.06024578629697393
epoch: 1, iteration: 30000, loss: 0.045469960672572994
epoch: 1, iteration: 40000, loss: 0.028724395240839713
epoch: 1, iteration: 50000, loss: 0.025964710871499326
epoch: 1, iteration: 60000, loss: 0.05065959929722678
epoch: 1, iteration: 70000, loss: 0.04772603486933481
epoch: 2, iteration: 0, loss: 0.058074384743321404
epoch: 2, iteration: 10000, loss: 0.047027338608539464
epoch: 2, iteration: 20000, loss:

KeyboardInterrupt: 

In [161]:
x = torch.stack([
    torch.linspace(90e6, 110e6, 1024),
    U*torch.ones(1024)
], 1).double().unsqueeze(0)

y = net(x)

In [163]:
y[0, :, 0]

tensor([ 0.6509,  0.6509,  0.6509,  ...,  0.6509,  0.6509,  0.6509], dtype=torch.float64)

In [165]:
targets.mean()

0.6495000000000004

In [105]:
loss.item()

0.1432942897081375

In [166]:
print(net)

Net(
  (lin1): Linear(in_features=2, out_features=64, bias=True)
  (lin2): Linear(in_features=64, out_features=1, bias=True)
)
